imports

In [2]:
import numpy as np
import pandas as pd

## Pandas - `lambda df:` 의 활용

### 추후 작성

## Pandas - MultiIndex의 이해

### 추후 작성

## Pandas - tidydata의 개념

- 아래의 자료는 처리가 불편하다.

In [15]:
df = pd.read_csv("https://raw.githubusercontent.com/guebin/DV2022/master/posts/Simpson.csv",index_col=0,header=[0,1])\
.stack().stack().reset_index()\
.rename({'level_0':'department','level_1':'result','level_2':'gender',0:'count'},axis=1).pivot_table(index=['gender','department'], columns='result',values='count',aggfunc=sum)
df 

result             fail  pass
gender department            
female A             19    89
       B              7    18
       C            391   202
       D            244   131
       E            299    94
       F            103   238
male   A            314   511
       B            208   352
       C            204   121
       D            279   138
       E            137    54
       F            149   224

`-` 가정1: 만약에 A학과에 해당하는 결과만 뽑고 싶다면? → department가 column으로 있어야함..

`-` 가정2: 이 데이터를 바탕으로 합격한사람만 bar plot을 그리고 싶다면? → department, gender, pass 가 column으로 있어야함..

In [16]:
tidydata = df['pass'].reset_index()
tidydata

,gender,department,pass
0,female,A,89
1,female,B,18
2,female,C,202
3,female,D,131
4,female,E,94
5,female,F,238
6,male,A,511
7,male,B,352
8,male,C,121
9,male,D,138


In [18]:
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

In [31]:
tidydata.plot.bar(
    x='gender', y='pass',
    color=["red", "blue"],
    facet_col ='department'
)

ValueError: All arguments should have the same length. The length of argument `color` is 2, whereas the length of  previously-processed arguments ['gender', 'pass', 'department'] is 12